# Scrape Magic Links

In [16]:
from splinter import Browser
from bs4 import BeautifulSoup as soup
from selenium.webdriver.chrome.service import Service
import time
from pymongo import MongoClient

In [17]:
mongo = MongoClient()
db = mongo['magic']
card_decks = db['card_decks']

In [18]:
my_service = Service(executable_path='c:/chrome_driver/chromedriver.exe')
browser = Browser('chrome', service=my_service)

In [19]:
def read_cards(url):
    cards_col = db['cards']
    my_service = Service(executable_path='c:/chrome_driver/chromedriver.exe')
    card_browser = Browser('chrome', service=my_service)
    card_browser.visit(url)
    time.sleep(1)
    card_page = soup(card_browser.html)
    card_list = []
    for card in card_page.find('div',class_='maindeck').find_all('li','list__item'):
        the_card =card.find('span',class_='').text.strip()
        card_list.append(the_card)
        cards_col.update_one({"card":the_card}, {"$set":{"card":the_card,"multiple":True}}, upsert=True)
    card_browser.quit()
    return card_list

In [20]:
# Visit the website
url = 'https://infinite.tcgplayer.com/magic-the-gathering/decks/advanced-search?formats=standard&p=1'
browser.visit(url)

In [21]:
# Create a BeautifulSoup parser to parse the HTML codepage_s
page_soup = soup(browser.html)

In [22]:
# find all of the rows
title = page_soup.find('div', class_='grid').find('a', class_='martech-base-link').find('h2',class_='title')
price = page_soup.find('div', class_='grid').find('a', class_='martech-base-link').find('div',class_='market-price')
xurl = page_soup.find('div', class_='grid').find('a', class_='martech-base-link')['href']
deck_base_url ='https://infinite.tcgplayer.com'
print(title, price, xurl)

<h2 class="title martech-text-lg martech-text-bold martech-barlow martech-one-line" data-testid="grid-card__card-grid-deck__title" data-v-45c68ff8="">Esper Legends <!-- --></h2> <div class="market-price" data-testid="RouterLink__market-price-template" data-v-df6ff770=""><div class="market-price__new-style" data-v-df6ff770=""><p class="martech-text-sm" data-v-df6ff770="">$544.48</p></div></div> /magic-the-gathering/deck/Esper-Legends/478732


In [23]:
# Extract and store the cards for each of the decks 
browser.visit(url)
for i in range(1,2):
    print(f"getting page {i} of 1")
    time.sleep(2)
    page_soup = soup(browser.html)
   
    result_grid = page_soup.find('div', class_='grid').find_all('a', class_='martech-base-link')
    deck ={}
    for item in result_grid:
        try:
            deck ={}
            deck['title'] = item.find('h2',class_='title').text.strip()
            deck['price'] = item.find('div',class_='market-price').text.strip()
            deck['url'] = item['href']
            card_url = deck_base_url + deck['url'] 
            deck['cards'] = read_cards(card_url)
            card_decks.insert_one(deck)
        except AttributeError as e:
            pass
    browser.find_by_css('.martech-pagination-next').click()

getting page 1 of 1


ElementClickInterceptedException: Message: element click intercepted: Element <span data-v-2100c89d="" data-v-fce9b973="" tag="button" class="martech-pagination-next" disabled="false" aria-label="Next Page">...</span> is not clickable at point (673, 403). Other element would receive the click: <div data-v-0313651e="" class="subscribe-drawer--columns">...</div>
  (Session info: chrome=117.0.5938.150)
Stacktrace:
	GetHandleVerifier [0x00007FF7ED927892+54818]
	(No symbol) [0x00007FF7ED896AC2]
	(No symbol) [0x00007FF7ED74DA3B]
	(No symbol) [0x00007FF7ED7947CB]
	(No symbol) [0x00007FF7ED792B99]
	(No symbol) [0x00007FF7ED790968]
	(No symbol) [0x00007FF7ED78FA23]
	(No symbol) [0x00007FF7ED78571F]
	(No symbol) [0x00007FF7ED7AEAAA]
	(No symbol) [0x00007FF7ED785036]
	(No symbol) [0x00007FF7ED7AECC0]
	(No symbol) [0x00007FF7ED7C75A2]
	(No symbol) [0x00007FF7ED7AE883]
	(No symbol) [0x00007FF7ED783691]
	(No symbol) [0x00007FF7ED7848D4]
	GetHandleVerifier [0x00007FF7EDC8B992+3610402]
	GetHandleVerif

## Scrape Book Links from Multiple Pages

In [ ]:
# Automate clicking the next button
import pandas as pd
pd.DataFrame(decks)
#martech-pagination-next


In [ ]:
card_page = soup(browser.html)
card_page.find('div',class_='maindeck').find('li','list__item').find('span',class_='').text

In [ ]:
browser.visit(deck_base_url)

In [ ]:
browser.quit()